In [1]:
# Install required libraries
%pip install torch torchaudio transformers datasets[audio] scikit-learn numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import required libraries
import torch

from datasets import load_dataset
from transformers import ClapModel, ClapProcessor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# Load dataset
audio_dataset = load_dataset('TrainingDataPro/speech-emotion-recognition-dataset', split='train')

# Load CLAP model
model = ClapModel.from_pretrained('laion/larger_clap_music').to('cuda:0')
processor = ClapProcessor.from_pretrained('laion/larger_clap_music')

# Preprocess audio samples
emotions = ['euphoric', 'joyfully', 'sad', 'surprised']
sample_inputs = []
audio_embeddings = []
targets = []

SAMPLING_RATE = 48000

for emotion in emotions:
    for sample in audio_dataset[emotion]:
        sample_inputs.append(
            processor(
                audios=sample['array'],
                sampling_rate=SAMPLING_RATE,
                return_tensors='pt'
            ).to('cuda:0')
        )
        targets.append(emotion)

for inputs in sample_inputs:
    audio_embeddings.append(model.get_audio_features(**inputs))
    
torch.save(audio_embeddings, 'audio_embeddings.pt')

In [4]:
# Flatten and process embeddings
flattened_embeddings = [embedding.view(-1) for embedding in audio_embeddings]
embedding_matrix = torch.stack(flattened_embeddings)

# Convert embeddings from tensor to numpy arrays
embedding_matrix = embedding_matrix.detach().cpu().numpy()

# Scale embedding data
scaler = StandardScaler()
embedding_matrix = scaler.fit_transform(embedding_matrix)
print(embedding_matrix.shape)
print(embedding_matrix)

(80, 512)
[[-1.3522571  -0.05475908 -0.6743673  ...  0.6996808   0.65430874
  -0.5138837 ]
 [ 0.50268257  0.63566035  3.7786016  ...  0.38104382  2.0940197
   0.65663946]
 [ 0.7463173   0.5486661   0.67125165 ... -1.7268778  -1.084867
   0.5785801 ]
 ...
 [ 0.2954817   0.2381683  -0.16123635 ...  0.42446738 -1.0245465
   0.60514367]
 [-0.22036797 -0.13910376 -0.90836185 ...  0.9615125   0.09734965
   0.3090269 ]
 [ 2.0981119  -0.27093282  0.71846426 ... -1.3601024  -0.06620956
   1.1445159 ]]


In [5]:
# TO-DO - STILL BROKEN
# Train Naive-Bayes classifier
train_embeddings = audio_embeddings[0:63]
print(len(train_embeddings))

model = GaussianNB()
model.fit()

train_pred = model.predict(embedding_matrix)
test_pred = model.predict(embedding_matrix)
train_accuracy = accuracy_score(train_pred, targets)

print(train_accuracy)

63


TypeError: GaussianNB.fit() missing 2 required positional arguments: 'X' and 'y'